In [245]:
import pandas as pd
from part3 import *
import plotly.graph_objects as go

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Load Data

In [220]:
SABR_Alpha = pd.read_pickle("SABR_Alpha_full.pickle")
SABR_Nu = pd.read_pickle("SABR_Nu_full.pickle")
SABR_Rho = pd.read_pickle("SABR_Rho_full.pickle")
ois = pd.read_csv("ois_3m.csv")

# Forward swap rates with 6 month payment frequency
forward_swap_rates = pd.read_csv("forward_swap_rates_all_years.csv")
forward_swap_rates = forward_swap_rates.pivot(index='forward', columns='tenor')['forward_swap_rate']

# Forward swap rates with 3 month payment frequency
forward_swap_rates_3m = pd.read_csv("forward_swap_6m_rates_3m.csv")
forward_swap_rates_3m = forward_swap_rates_3m.pivot(index='forward', columns='tenor')['forward_swap_rate']

#### CMS calculations

CMS 10y with semi-annual payment over next 5 years

In [178]:
PriceCMSLeg(ois=ois, 
            forward_swap_rates=forward_swap_rates,
            SABR_Alpha=SABR_Alpha,
            SABR_Rho=SABR_Rho,
            SABR_Nu=SABR_Nu,
            CMS_Maturity=5,
            N=10,
            m=0.5)

0.22261816217721075

In [222]:
PriceCMSLeg(ois=ois, 
            forward_swap_rates=forward_swap_rates_3m,
            SABR_Alpha=SABR_Alpha,
            SABR_Rho=SABR_Rho,
            SABR_Nu=SABR_Nu,
            CMS_Maturity=10,
            N=2,
            m=0.25)

0.5335723868598041

#### Effect of Maturity and Tenor on Convexity Correction

In [243]:
maturities = [1, 5, 10]
tenors = [1, 2, 3, 5, 10]
cms_rates = compute_cms_rates(maturities=maturities, 
                 tenors=tenors, 
                 m=0.5, 
                 forward_swap_rates=forward_swap_rates,
                 SABR_Alpha=SABR_Alpha,
                 SABR_Rho=SABR_Rho,
                 SABR_Nu=SABR_Nu)

cms_rates

Tenor,1,2,3,5,10
Maturity,,,,,
1,0.033304,0.034871,0.035466,0.036266,0.039722
5,0.056573,0.054324,0.052156,0.048488,0.051908
10,0.070696,0.074424,0.076655,0.072393,0.086971


In [244]:
comparable_swaps = forward_swap_rates.copy(deep=True)
comparable_swaps.index.name = "Maturity"
comparable_swaps.columns.name = "Tenor"
comparable_swaps = comparable_swaps.loc[maturities, tenors]
comparable_swaps

Tenor,1,2,3,5,10
Maturity,,,,,
1,0.032006,0.033258,0.034010,0.035255,0.038423
5,0.039273,0.040074,0.040069,0.041089,0.043619
10,0.042151,0.043078,0.044062,0.046210,0.053405


In [257]:
convexity_correction = cms_rates - comparable_swaps

In [260]:
fig = go.Figure()
fig.add_trace(go.Surface(z=convexity_correction.values, x=convexity_correction.columns.values, y=convexity_correction.index.values))
fig.update_layout(title="Convexity Correction (z) vs Tenor (x) vs Maturity (y)", scene=dict(xaxis_title="Tenor", yaxis_title="Maturity"), height=700)
fig.show()